<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/limitspollutants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

############################################################
# AIR QUALITY EXCEEDANCES – DETAILED LIST + ANNUAL SUMMARY
# GOOGLE COLAB VERSION (PYTHON) – CORRECTED
############################################################

# 1. Install and load libraries
!pip install pandas numpy -q

import pandas as pd
import numpy as np
import zipfile
import io
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")

# 2. Upload the CSV file
print("⬆️ Please upload your CSV file (processed_data_wide.csv):")
uploaded = files.upload()
if not uploaded:
    raise ValueError("❌ No file uploaded. Run the cell again.")
filename = list(uploaded.keys())[0]
print(f"📁 Uploaded file: {filename}")

# 3. Read data
df = pd.read_csv(io.BytesIO(uploaded[filename]))
if 'date' not in df.columns:
    raise ValueError("❌ Column 'date' not found.")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# 4. Analysis period (1991-2021)
START_YEAR = 1991
END_YEAR   = 2021
df = df[(df['date'].dt.year >= START_YEAR) & (df['date'].dt.year <= END_YEAR)].copy()
print(f"📊 Analysing period: {START_YEAR} – {END_YEAR} ({len(df)} rows)")

# 5. Define EU and OMS 2024 limits with allowed exceedances per year
#    Structure: {pollutant: {period: (threshold, allowed_exceedances)}}
#    For annual period, allowed_exceedances = None (we compare annual mean directly to threshold)
limits_eu = {
    'no2':   {'hour': (200, 18),   'year': (40, None)},
    'pm10':  {'day':  (50, 35),    'year': (40, None)},
    'pm2.5': {'year': (25, None)},                     # EU has no daily limit
    'so2':   {'hour': (350, 24),   'day': (125, 3)},
    'o3':    {'max8h': (120, 25)},                     # allowed exceedances per year (averaged over 3 years, but we use per year)
    'co':    {'max8h': (10, 0)},                        # mg/m³, no allowed exceedances
    'c6h6':  {'year': (5, None)},
    'pb':    {'year': (0.5, None)},
    'as':    {'year': (0.006, None)},
    'cd':    {'year': (0.005, None)},
    'ni':    {'year': (0.020, None)},
    'ba_p':  {'year': (0.001, None)}
}

limits_oms = {
    'no2':   {'day':  (25, 4),      'year': (10, None)},
    'pm10':  {'day':  (45, 4),      'year': (15, None)},
    'pm2.5': {'day':  (15, 4),      'year': (5, None)},
    'so2':   {'day':  (40, 4)},
    'o3':    {'max8h': (100, 0)},
    'co':    {'max8h': (4, 0)},
    'c6h6':  {'year': (1.7, None)},                     # WHO 2010
    'pb':    {'year': (0.5, None)},
    'as':    {'year': (0.0066, None)},
    'cd':    {'year': (0.005, None)},
    'ni':    {'year': (0.025, None)},
    'ba_p':  {'year': (0.00012, None)}
}

# 6. Helper functions
def exists(col):
    return col in df.columns

def detect_periods(poll):
    """Return set of periods that exist in either EU or WHO limits."""
    periods = set()
    if poll in limits_eu:
        periods.update(limits_eu[poll].keys())
    if poll in limits_oms:
        periods.update(limits_oms[poll].keys())
    return periods

def get_thresholds(poll, period):
    """Return (eu_thresh, eu_allowed, who_thresh, who_allowed) or (None, None, None, None)."""
    eu = limits_eu.get(poll, {}).get(period, (None, None))
    who = limits_oms.get(poll, {}).get(period, (None, None))
    return eu[0], eu[1], who[0], who[1]

def format_date(row):
    if row['period'] == 'year':
        return row['date'].strftime('%Y')
    elif row['period'] == 'day':
        return row['date'].strftime('%Y-%m-%d')
    else:   # hour, max8h
        return row['date'].strftime('%Y-%m-%d %H:%M:%S')

# 7. Generate detailed list of exceedances
exceedances_list = []

for poll in set(limits_eu.keys()) | set(limits_oms.keys()):
    if not exists(poll):
        continue
    periods = detect_periods(poll)

    for period in periods:
        eu_th, eu_all, who_th, who_all = get_thresholds(poll, period)

        if period == 'hour':
            data = df[['date', poll]].copy()
            if eu_th is not None:
                mask = data[poll] > eu_th
                if mask.any():
                    tmp = data[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'hour'
                    tmp['value'] = tmp[poll]
                    tmp['threshold_eu'] = eu_th
                    tmp['threshold_who'] = who_th if who_th is not None else np.nan
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])
            if who_th is not None and (eu_th is None or who_th != eu_th):
                mask = data[poll] > who_th
                if mask.any():
                    tmp = data[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'hour'
                    tmp['value'] = tmp[poll]
                    tmp['threshold_eu'] = eu_th if eu_th is not None else np.nan
                    tmp['threshold_who'] = who_th
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])

        elif period == 'day':
            daily = df.set_index('date').resample('D')[poll].mean().reset_index()
            daily = daily.rename(columns={poll: 'value'})
            if eu_th is not None:
                mask = daily['value'] > eu_th
                if mask.any():
                    tmp = daily[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'day'
                    tmp['threshold_eu'] = eu_th
                    tmp['threshold_who'] = who_th if who_th is not None else np.nan
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])
            if who_th is not None and (eu_th is None or who_th != eu_th):
                mask = daily['value'] > who_th
                if mask.any():
                    tmp = daily[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'day'
                    tmp['threshold_eu'] = eu_th if eu_th is not None else np.nan
                    tmp['threshold_who'] = who_th
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])

        elif period == 'max8h':
            df_sorted = df.sort_values('date').copy()
            rolling = df_sorted[poll].rolling(window=8, min_periods=6).mean()
            exceed_mask = pd.Series(False, index=df_sorted.index)
            if eu_th is not None:
                exceed_mask |= (rolling > eu_th)
            if who_th is not None:
                exceed_mask |= (rolling > who_th)
            if exceed_mask.any():
                tmp = df_sorted.loc[exceed_mask, ['date']].copy()
                tmp['value'] = rolling[exceed_mask].values
                tmp['pollutant'] = poll
                tmp['period'] = 'max8h'
                tmp['threshold_eu'] = eu_th if eu_th is not None else np.nan
                tmp['threshold_who'] = who_th if who_th is not None else np.nan
                exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])

        elif period == 'year':
            yearly = df.groupby(df['date'].dt.year)[poll].mean().reset_index()
            yearly = yearly.rename(columns={'date': 'year', poll: 'value'})
            yearly['date'] = pd.to_datetime(yearly['year'].astype(str) + '-01-01')
            if eu_th is not None:
                mask = yearly['value'] > eu_th
                if mask.any():
                    tmp = yearly[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'year'
                    tmp['threshold_eu'] = eu_th
                    tmp['threshold_who'] = who_th if who_th is not None else np.nan
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])
            if who_th is not None and (eu_th is None or who_th != eu_th):
                mask = yearly['value'] > who_th
                if mask.any():
                    tmp = yearly[mask].copy()
                    tmp['pollutant'] = poll
                    tmp['period'] = 'year'
                    tmp['threshold_eu'] = eu_th if eu_th is not None else np.nan
                    tmp['threshold_who'] = who_th
                    exceedances_list.append(tmp[['date','pollutant','period','value','threshold_eu','threshold_who']])

if exceedances_list:
    detailed = pd.concat(exceedances_list, ignore_index=True)
    detailed = detailed.sort_values(['date', 'pollutant', 'period'])
    # Format date
    detailed['date_str'] = detailed.apply(format_date, axis=1)
    detailed = detailed[['pollutant', 'period', 'date_str', 'value', 'threshold_eu', 'threshold_who']]
else:
    detailed = pd.DataFrame(columns=['pollutant','period','date_str','value','threshold_eu','threshold_who'])

# 8. Generate annual summary
summary_rows = []

for poll in set(limits_eu.keys()) | set(limits_oms.keys()):
    if not exists(poll):
        continue
    periods = detect_periods(poll)

    for period in periods:
        eu_th, eu_all, who_th, who_all = get_thresholds(poll, period)

        if period == 'year':
            yearly = df.groupby(df['date'].dt.year)[poll].mean()
            for year, val in yearly.items():
                if pd.isna(val):
                    continue
                eu_comp = val <= eu_th if eu_th is not None else None
                who_comp = val <= who_th if who_th is not None else None
                summary_rows.append({
                    'Year': year,
                    'Pollutant': poll,
                    'Period': period,
                    'Observed_EU': round(val, 3),
                    'Allowed_EU': eu_th if eu_th is not None else '-',
                    'EU_compliance': eu_comp,
                    'Observed_WHO': round(val, 3),
                    'Allowed_WHO': who_th if who_th is not None else '-',
                    'WHO_compliance': who_comp
                })
        else:
            df['year'] = df['date'].dt.year
            years = df['year'].unique()
            for year in years:
                data_year = df[df['year'] == year]
                # Observed using EU threshold
                if eu_th is not None:
                    if period == 'hour':
                        obs_eu = (data_year[poll] > eu_th).sum()
                    elif period == 'day':
                        daily = data_year.set_index('date').resample('D')[poll].mean()
                        obs_eu = (daily > eu_th).sum()
                    elif period == 'max8h':
                        data_sorted = data_year.sort_values('date')
                        rolling = data_sorted[poll].rolling(window=8, min_periods=6).mean()
                        obs_eu = (rolling > eu_th).sum()
                    else:
                        obs_eu = None
                else:
                    obs_eu = None

                # Observed using WHO threshold
                if who_th is not None:
                    if period == 'hour':
                        obs_who = (data_year[poll] > who_th).sum()
                    elif period == 'day':
                        daily = data_year.set_index('date').resample('D')[poll].mean()
                        obs_who = (daily > who_th).sum()
                    elif period == 'max8h':
                        data_sorted = data_year.sort_values('date')
                        rolling = data_sorted[poll].rolling(window=8, min_periods=6).mean()
                        obs_who = (rolling > who_th).sum()
                    else:
                        obs_who = None
                else:
                    obs_who = None

                # Compliance
                eu_comp = (obs_eu <= eu_all) if (eu_th is not None and eu_all is not None and obs_eu is not None) else None
                who_comp = (obs_who <= who_all) if (who_th is not None and who_all is not None and obs_who is not None) else None

                summary_rows.append({
                    'Year': year,
                    'Pollutant': poll,
                    'Period': period,
                    'Observed_EU': obs_eu if obs_eu is not None else '-',
                    'Allowed_EU': eu_all if eu_all is not None else '-',
                    'EU_compliance': eu_comp,
                    'Observed_WHO': obs_who if obs_who is not None else '-',
                    'Allowed_WHO': who_all if who_all is not None else '-',
                    'WHO_compliance': who_comp
                })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(['Year', 'Pollutant', 'Period'])

# 9. Save both CSVs and create ZIP
csv_list = "exceedances_list.csv"
csv_summary = "exceedances_summary.csv"
zip_name = "air_quality_results.zip"

detailed.to_csv(csv_list, index=False)
summary_df.to_csv(csv_summary, index=False)

with zipfile.ZipFile(zip_name, 'w') as zipf:
    zipf.write(csv_list)
    zipf.write(csv_summary)

files.download(zip_name)

# 10. Print preview
print("✅ Processing complete. ZIP file downloaded.")
print("\n📋 Detailed exceedances preview (first 10 rows):")
print(detailed.head(10).to_string(index=False))
print("\n📋 Annual summary preview (first 10 rows):")
print(summary_df.head(10).to_string(index=False))

############################################################
# END OF CELL
############################################################

✅ Libraries loaded
⬆️ Please upload your CSV file (processed_data_wide.csv):


Saving processed_data_wide.csv to processed_data_wide (1).csv
📁 Uploaded file: processed_data_wide (1).csv
📊 Analysing period: 1991 – 2021 (255768 rows)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Processing complete. ZIP file downloaded.

📋 Detailed exceedances preview (first 10 rows):
pollutant period   date_str     value  threshold_eu  threshold_who
      no2   year       1991 41.993014          40.0           10.0
      no2   year       1991 41.993014          40.0           10.0
      no2    day 1991-11-26 71.045455           NaN           25.0
      so2    day 1991-11-26 63.083333         125.0           40.0
      no2    day 1991-11-27 89.916667           NaN           25.0
      so2    day 1991-11-27 43.000000         125.0           40.0
      no2    day 1991-11-28 75.416667           NaN           25.0
      so2    day 1991-11-28 49.500000         125.0           40.0
      no2    day 1991-11-29 75.416667           NaN           25.0
      no2    day 1991-11-30 55.875000           NaN           25.0

📋 Annual summary preview (first 10 rows):
 Year Pollutant Period Observed_EU Allowed_EU EU_compliance Observed_WHO Allowed_WHO WHO_compliance
 1991        co  max8h     

In [ ]:

############################################################
# AIR QUALITY EXCEEDANCES SUMMARY (EU + OMS 2024)
# GOOGLE COLAB VERSION (PYTHON)
############################################################

# 1. Install and load libraries
!pip install pandas numpy -q

import pandas as pd
import numpy as np
import zipfile
import io
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")

# 2. Upload the CSV file
print("⬆️ Please upload your CSV file (processed_data_wide.csv):")
uploaded = files.upload()
if not uploaded:
    raise ValueError("❌ No file uploaded. Run the cell again.")
filename = list(uploaded.keys())[0]
print(f"📁 Uploaded file: {filename}")

# 3. Read data
df = pd.read_csv(io.BytesIO(uploaded[filename]))
if 'date' not in df.columns:
    raise ValueError("❌ Column 'date' not found.")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# 4. Analysis period (1991-2021)
START_YEAR = 1991
END_YEAR   = 2021
df = df[(df['date'].dt.year >= START_YEAR) & (df['date'].dt.year <= END_YEAR)].copy()
print(f"📊 Analysing period: {START_YEAR} – {END_YEAR} ({len(df)} rows)")

# 5. Define EU and OMS 2024 limits with allowed exceedances per year
#    Structure: {pollutant: {period: (threshold, allowed_exceedances)}}
#    For annual period, allowed_exceedances = None (we compare annual mean directly to threshold)
limits_eu = {
    'no2':   {'hour': (200, 18),   'year': (40, None)},
    'pm10':  {'day':  (50, 35),    'year': (40, None)},
    'pm2.5': {'year': (25, None)},                     # EU has no daily limit
    'so2':   {'hour': (350, 24),   'day': (125, 3)},
    'o3':    {'max8h': (120, 25)},                     # allowed exceedances per year (averaged over 3 years, but we use per year)
    'co':    {'max8h': (10, 0)},                        # mg/m³, no allowed exceedances
    'c6h6':  {'year': (5, None)},
    'pb':    {'year': (0.5, None)},
    'as':    {'year': (0.006, None)},
    'cd':    {'year': (0.005, None)},
    'ni':    {'year': (0.020, None)},
    'ba_p':  {'year': (0.001, None)}
}

limits_oms = {
    'no2':   {'day':  (25, 4),      'year': (10, None)},
    'pm10':  {'day':  (45, 4),      'year': (15, None)},
    'pm2.5': {'day':  (15, 4),      'year': (5, None)},
    'so2':   {'day':  (40, 4)},
    'o3':    {'max8h': (100, 0)},
    'co':    {'max8h': (4, 0)},
    'c6h6':  {'year': (1.7, None)},                     # WHO 2010
    'pb':    {'year': (0.5, None)},
    'as':    {'year': (0.0066, None)},
    'cd':    {'year': (0.005, None)},
    'ni':    {'year': (0.025, None)},
    'ba_p':  {'year': (0.00012, None)}
}

# 6. Helper functions to compute exceedance counts per year
def exists(col):
    return col in df.columns

def count_exceedances(series, threshold):
    """Count number of exceedances of a threshold in a pandas Series (hourly or 8‑hour values)."""
    return (series > threshold).sum()

def count_exceedances_daily(col, threshold):
    """Count number of days where daily mean exceeds threshold."""
    if not exists(col): return None
    daily = df.set_index('date').resample('D')[col].mean()
    return (daily > threshold).sum()

def count_exceedances_8h(col, threshold):
    """Count number of 8‑hour rolling means exceeding threshold (using at least 6 values)."""
    if not exists(col): return None
    df_sorted = df.sort_values('date').copy()
    rolling = df_sorted[col].rolling(window=8, min_periods=6).mean()
    return (rolling > threshold).sum()

def annual_mean(col):
    if not exists(col): return None
    return df.groupby(df['date'].dt.year)[col].mean()

def annual_exceedances(period_func, col, threshold, allowed):
    """Return Series with index year and values (observed, allowed)."""
    if not exists(col):
        return None
    if period_func == 'year':
        # annual mean
        obs = annual_mean(col)
        return obs, None  # allowed not applicable for annual mean
    else:
        # count exceedances per year
        df['year'] = df['date'].dt.year
        counts = []
        years = []
        for year, grp in df.groupby('year'):
            if period_func == 'hour':
                cnt = count_exceedances(grp[col], threshold)
            elif period_func == 'day':
                # we need daily means per year
                daily_grp = grp.set_index('date').resample('D')[col].mean()
                cnt = (daily_grp > threshold).sum()
            elif period_func == 'max8h':
                # compute 8h means for this year's data
                grp_sorted = grp.sort_values('date')
                rolling = grp_sorted[col].rolling(window=8, min_periods=6).mean()
                cnt = (rolling > threshold).sum()
            else:
                cnt = None
            years.append(year)
            counts.append(cnt)
        obs = pd.Series(counts, index=years)
        return obs, allowed

# 7. Build summary dataframe
summary_rows = []

# Get all unique years present
years_present = sorted(df['date'].dt.year.unique())

# For each pollutant defined in EU (or OMS, we can iterate over union)
all_pollutants = set(limits_eu.keys()) | set(limits_oms.keys())

for poll in all_pollutants:
    # EU data
    eu_periods = limits_eu.get(poll, {})
    oms_periods = limits_oms.get(poll, {})

    # Union of periods
    periods = set(eu_periods.keys()) | set(oms_periods.keys())

    for period in periods:
        # Get EU parameters
        eu_thresh, eu_allowed = eu_periods.get(period, (None, None))
        oms_thresh, oms_allowed = oms_periods.get(period, (None, None))

        # Skip if pollutant column missing for both? We'll compute only if column exists.
        if not exists(poll):
            continue

        # For each year, compute observed counts/means
        if period == 'year':
            # Annual mean
            obs_series = annual_mean(poll)
            if obs_series is None:
                continue
            for year, obs_val in obs_series.items():
                if pd.isna(obs_val):
                    continue
                # EU compliance
                if eu_thresh is not None:
                    eu_comp = obs_val <= eu_thresh
                else:
                    eu_comp = None
                # WHO compliance
                if oms_thresh is not None:
                    who_comp = obs_val <= oms_thresh
                else:
                    who_comp = None
                summary_rows.append({
                    'Year': year,
                    'Pollutant': poll,
                    'Period': period,
                    'Observed': round(obs_val, 3),
                    'Allowed_EU': eu_thresh if eu_thresh is not None else '-',
                    'EU_compliance': eu_comp,
                    'Allowed_WHO': oms_thresh if oms_thresh is not None else '-',
                    'WHO_compliance': who_comp
                })
        else:
            # Non-annual: count exceedances
            # We need to compute observed counts using EU threshold? Actually for EU compliance we need counts based on EU threshold.
            # For WHO compliance we need counts based on WHO threshold.
            # So we might need two separate observed counts. But the user wants a single observed column? Possibly they want the count of exceedances based on EU threshold (the legal standard). But to decide WHO compliance we need to compare with WHO threshold.
            # We'll compute observed count using EU threshold, and also separately compute count using WHO threshold if needed.
            # However, the observed column could be the count using the EU threshold, as that's the one used for EU compliance.
            # For WHO, we need to compare a count (using WHO threshold) with WHO allowed.
            # So we need two observed counts? The user didn't specify. Let's provide two observed columns: Observed_EU and Observed_WHO? That might be cluttered.
            # Alternative: For each standard, we compute compliance based on the respective threshold and allowed.
            # We'll keep a single observed column representing the count using the EU threshold (the stricter? not necessarily). For WHO, we'll compute compliance using the same observed counts? That would be incorrect because WHO uses a different threshold. So we need two separate observed counts.
            # Therefore we'll produce two observed columns: Observed_EU (counts using EU threshold) and Observed_WHO (counts using WHO threshold).
            # That fits the idea of "number of exceedances observed" per standard.
            # Then Allowed_EU and Allowed_WHO are the permitted numbers.
            # Compliance is then True if Observed_EU <= Allowed_EU (if Allowed_EU is not None) else based on annual mean? Already handled.
            # For periods where a standard does not define a limit, we leave as NA.
            # So modify summary rows accordingly.

            # Compute observed counts per year for both thresholds
            obs_eu_series = None
            obs_who_series = None
            if eu_thresh is not None:
                if period == 'hour':
                    obs_eu_series = df.groupby(df['date'].dt.year)[poll].apply(lambda x: count_exceedances(x, eu_thresh))
                elif period == 'day':
                    obs_eu_series = df.groupby(df['date'].dt.year).apply(lambda g: count_exceedances_daily(g, poll, eu_thresh))
                elif period == 'max8h':
                    obs_eu_series = df.groupby(df['date'].dt.year).apply(lambda g: count_exceedances_8h(g, poll, eu_thresh))
            if oms_thresh is not None:
                if period == 'hour':
                    obs_who_series = df.groupby(df['date'].dt.year)[poll].apply(lambda x: count_exceedances(x, oms_thresh))
                elif period == 'day':
                    obs_who_series = df.groupby(df['date'].dt.year).apply(lambda g: count_exceedances_daily(g, poll, oms_thresh))
                elif period == 'max8h':
                    obs_who_series = df.groupby(df['date'].dt.year).apply(lambda g: count_exceedances_8h(g, poll, oms_thresh))

            # Combine years from both series
            all_years = set()
            if obs_eu_series is not None:
                all_years.update(obs_eu_series.index)
            if obs_who_series is not None:
                all_years.update(obs_who_series.index)
            all_years = sorted(all_years)

            for year in all_years:
                obs_eu = obs_eu_series.get(year, np.nan) if obs_eu_series is not None else np.nan
                obs_who = obs_who_series.get(year, np.nan) if obs_who_series is not None else np.nan

                # EU compliance
                if eu_thresh is not None and eu_allowed is not None and not pd.isna(obs_eu):
                    eu_comp = obs_eu <= eu_allowed
                else:
                    eu_comp = None

                # WHO compliance
                if oms_thresh is not None and oms_allowed is not None and not pd.isna(obs_who):
                    who_comp = obs_who <= oms_allowed
                else:
                    who_comp = None

                summary_rows.append({
                    'Year': year,
                    'Pollutant': poll,
                    'Period': period,
                    'Observed_EU': round(obs_eu, 1) if not pd.isna(obs_eu) else '-',
                    'Allowed_EU': eu_allowed if eu_allowed is not None else '-',
                    'EU_compliance': eu_comp,
                    'Observed_WHO': round(obs_who, 1) if not pd.isna(obs_who) else '-',
                    'Allowed_WHO': oms_allowed if oms_allowed is not None else '-',
                    'WHO_compliance': who_comp
                })

# Convert to DataFrame
summary_df = pd.DataFrame(summary_rows)

# Sort
summary_df = summary_df.sort_values(['Year', 'Pollutant', 'Period'])

# 8. Save to CSV and ZIP
csv_name = "exceedances_summary_1991_2021.csv"
summary_df.to_csv(csv_name, index=False)

zip_name = "air_quality_summary.zip"
with zipfile.ZipFile(zip_name, 'w') as zipf:
    zipf.write(csv_name)

files.download(zip_name)

# 9. Print preview
print("✅ Processing complete. ZIP file downloaded.")
print("\n📋 Summary preview (first 20 rows):")
print(summary_df.head(20).to_string(index=False))

############################################################
# END OF CELL
############################################################

In [1]:

############################################################
# AIR QUALITY LIMITS CHECK (EU + OMS 2024)
# GOOGLE COLAB VERSION (PYTHON)
############################################################

# 1. Install and load libraries
!pip install pandas numpy -q

import pandas as pd
import numpy as np
import zipfile
import io
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")

# 2. Upload the CSV file
print("⬆️ Please upload your CSV file (processed_data_wide.csv):")
uploaded = files.upload()
if not uploaded:
    raise ValueError("❌ No file uploaded. Run the cell again.")
filename = list(uploaded.keys())[0]
print(f"📁 Uploaded file: {filename}")

# 3. Read data
df = pd.read_csv(io.BytesIO(uploaded[filename]))
if 'date' not in df.columns:
    raise ValueError("❌ Column 'date' not found.")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# 4. Analysis period (1991-2021)
START_YEAR = 1991
END_YEAR   = 2021
df = df[(df['date'].dt.year >= START_YEAR) & (df['date'].dt.year <= END_YEAR)].copy()
print(f"📊 Analysing period: {START_YEAR} – {END_YEAR} ({len(df)} rows)")

# 5. Define EU and OMS 2024 limits (μg/m³, except CO in mg/m³)
#    Period keys: 'hour', 'day', 'year', 'max8h' (8-hour running mean)
limits_eu = {
    'no2':   {'hour': 200, 'year': 40},
    'pm10':  {'day': 50,  'year': 40},
    'pm2.5': {'day': 25,  'year': 25},
    'so2':   {'hour': 350, 'day': 125, 'year': 20},
    'o3':    {'max8h': 120},
    'co':    {'max8h': 10},           # mg/m³
    'c6h6':  {'year': 5},              # benzene
    'pb':    {'year': 0.5},             # lead
    'as':    {'year': 0.006},           # arsenic (μg/m³, assuming data in μg/m³)
    'cd':    {'year': 0.005},           # cadmium
    'ni':    {'year': 0.020},           # nickel
    'ba_p':  {'year': 0.001}            # benzo(a)pyrene
}

limits_oms = {
    'no2':   {'day': 25,   'year': 10},
    'pm10':  {'day': 45,   'year': 15},
    'pm2.5': {'day': 15,   'year': 5},
    'so2':   {'day': 40},
    'o3':    {'max8h': 100},
    'co':    {'max8h': 4},              # mg/m³
    'c6h6':  {'year': 1.7},              # OMS 2010 (no specific 2024)
    'pb':    {'year': 0.5},               # same as EU
    'as':    {'year': 0.0066},            # approx
    'cd':    {'year': 0.005},
    'ni':    {'year': 0.025},
    'ba_p':  {'year': 0.00012}            # very low (0.12 ng/m³)
}

# 6. Helper functions for exceedances
def exists(col):
    return col in df.columns

def exceed_hour(col, limit):
    if not exists(col): return pd.DataFrame()
    tmp = df[df[col] > limit].copy()
    if tmp.empty: return pd.DataFrame()
    return pd.DataFrame({
        'pollutant': col,
        'period': 'hour',
        'date': tmp['date'],
        'value': tmp[col]
    })

def exceed_day(col, limit):
    if not exists(col): return pd.DataFrame()
    daily = df.set_index('date').resample('D')[col].mean().reset_index()
    daily = daily[daily[col] > limit].copy()
    if daily.empty: return pd.DataFrame()
    return pd.DataFrame({
        'pollutant': col,
        'period': 'day',
        'date': daily['date'],
        'value': daily[col]
    })

def exceed_year(col, limit):
    if not exists(col): return pd.DataFrame()
    df['year'] = df['date'].dt.year
    yearly = df.groupby('year')[col].mean().reset_index()
    yearly = yearly[yearly[col] > limit].copy()
    if yearly.empty: return pd.DataFrame()
    yearly['date'] = pd.to_datetime(yearly['year'].astype(str) + '-01-01')
    return pd.DataFrame({
        'pollutant': col,
        'period': 'year',
        'date': yearly['date'],
        'value': yearly[col]
    })

def exceed_max8h(col, limit):
    if not exists(col): return pd.DataFrame()
    df_sorted = df.sort_values('date').copy()
    rolling = df_sorted[col].rolling(window=8, min_periods=6).mean()
    exceed = df_sorted[rolling > limit].copy()
    if exceed.empty: return pd.DataFrame()
    return pd.DataFrame({
        'pollutant': col,
        'period': 'max8h',
        'date': exceed['date'],
        'value': rolling[exceed.index]
    })

# 7. Generate exceedances using EU limits (or OMS – you can change)
#    We'll use EU limits as reference for exceedances.
exceedances_list = []
for poll, periods in limits_eu.items():
    if not exists(poll):
        continue
    for per, lim in periods.items():
        if per == 'hour':
            dfp = exceed_hour(poll, lim)
        elif per == 'day':
            dfp = exceed_day(poll, lim)
        elif per == 'year':
            dfp = exceed_year(poll, lim)
        elif per == 'max8h':
            dfp = exceed_max8h(poll, lim)
        else:
            continue
        if not dfp.empty:
            exceedances_list.append(dfp)

exceedances = pd.concat(exceedances_list, ignore_index=True)

# 8. Format date according to period
def format_date(row):
    if row['period'] == 'year':
        return row['date'].strftime('%Y')
    elif row['period'] == 'day':
        return row['date'].strftime('%Y-%m-%d')
    else:   # hour, max8h
        return row['date'].strftime('%Y-%m-%d %H:%M:%S')

if not exceedances.empty:
    exceedances = exceedances.sort_values(['pollutant', 'period', 'date'])
    exceedances['date_str'] = exceedances.apply(format_date, axis=1)
    exceedances_out = exceedances[['pollutant', 'period', 'date_str', 'value']]
else:
    exceedances_out = pd.DataFrame(columns=['pollutant','period','date_str','value'])

# 9. Create limits tables
def build_limits_df(limits_dict, source_name):
    rows = []
    for poll, periods in limits_dict.items():
        for per, lim in periods.items():
            rows.append({
                'pollutant': poll,
                'period': per,
                'limit': lim,
                'unit': 'μg/m³' if poll != 'co' else 'mg/m³',
                'source': source_name
            })
    return pd.DataFrame(rows)

limits_eu_df = build_limits_df(limits_eu, 'EU')
limits_oms_df = build_limits_df(limits_oms, 'OMS 2024')

# 10. Comparison table (merge EU and OMS)
comparison = pd.merge(limits_eu_df, limits_oms_df,
                      on=['pollutant','period'],
                      suffixes=('_eu', '_oms'),
                      how='outer').fillna('-')
comparison = comparison[['pollutant','period','limit_eu','limit_oms','unit_eu','unit_oms','source_eu','source_oms']]

# 11. Save all to CSV and create ZIP
csv_exceed = "exceedances_1991_2021.csv"
csv_eu_lim  = "limits_eu.csv"
csv_oms_lim = "limits_oms.csv"
csv_comp    = "comparison_limits.csv"
zip_name    = "air_quality_results.zip"

exceedances_out.to_csv(csv_exceed, index=False)
limits_eu_df.to_csv(csv_eu_lim, index=False)
limits_oms_df.to_csv(csv_oms_lim, index=False)
comparison.to_csv(csv_comp, index=False)

with zipfile.ZipFile(zip_name, 'w') as zipf:
    zipf.write(csv_exceed)
    zipf.write(csv_eu_lim)
    zipf.write(csv_oms_lim)
    zipf.write(csv_comp)

files.download(zip_name)

# 12. Print summary
print("✅ Processing complete. ZIP file downloaded.")
print("\n📊 Exceedances summary (EU limits):")
if not exceedances.empty:
    summary = exceedances.groupby(['pollutant','period']).size().reset_index(name='count')
    print(summary.to_string(index=False))
else:
    print("No exceedances found.")

print("\n📋 EU limits table preview:")
print(limits_eu_df.head(10).to_string(index=False))
print("\n📋 OMS 2024 limits table preview:")
print(limits_oms_df.head(10).to_string(index=False))
print("\n📋 Comparison table preview:")
print(comparison.head(10).to_string(index=False))

############################################################
# END OF CELL
############################################################

✅ Libraries loaded
⬆️ Please upload your CSV file (processed_data_wide.csv):


Saving processed_data_wide.csv to processed_data_wide.csv
📁 Uploaded file: processed_data_wide.csv
📊 Analysing period: 1991 – 2021 (255768 rows)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Processing complete. ZIP file downloaded.

📊 Exceedances summary (EU limits):
pollutant period  count
      no2   hour     82
      no2   year     18
       o3  max8h    403
     pm10    day      9
      so2   hour      8
      so2   year      5

📋 EU limits table preview:
pollutant period  limit  unit source
      no2   hour  200.0 μg/m³     EU
      no2   year   40.0 μg/m³     EU
     pm10    day   50.0 μg/m³     EU
     pm10   year   40.0 μg/m³     EU
    pm2.5    day   25.0 μg/m³     EU
    pm2.5   year   25.0 μg/m³     EU
      so2   hour  350.0 μg/m³     EU
      so2    day  125.0 μg/m³     EU
      so2   year   20.0 μg/m³     EU
       o3  max8h  120.0 μg/m³     EU

📋 OMS 2024 limits table preview:
pollutant period  limit  unit   source
      no2    day   25.0 μg/m³ OMS 2024
      no2   year   10.0 μg/m³ OMS 2024
     pm10    day   45.0 μg/m³ OMS 2024
     pm10   year   15.0 μg/m³ OMS 2024
    pm2.5    day   15.0 μg/m³ OMS 2024
    pm2.5   year    5.0 μg/m³ OMS 2024
      so2 